# Introduction

Notebook to preprocess **bug reports** and **test cases** datasets and create from them the **oracle** dataset.

## Load Libraries and Data

In [87]:
import pandas as pd
import numpy as np
from sklearn.externals.joblib import Parallel, delayed
from tqdm import tqdm

from utils import aux_functions

testcases = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/TC/testcases.csv')
print('Test Cases Shape: {}'.format(testcases.shape))

bugreports_p1 = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/BR/all_bugs_p1.csv', sep="|")
bugreports_p2 = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/BR/all_bugs_p2.csv', sep='|')
bugreports_p3 = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/BR/all_bugs_p3.csv', sep='|')

bugreports = pd.concat([bugreports_p1, bugreports_p2, bugreports_p3])
print('Bug Reports shape: {}'.format(bugreports.shape))

Test Cases Shape: (207, 10)
Bug Reports shape: (35955, 10)


## BugReports

#### Removing Duplicate Bugs

In [88]:
print('BR previous shape: {}'.format(bugreports.shape))
bugreports.drop_duplicates('Bug_Number', keep=False, inplace=True)
print('BR shape: {}'.format(bugreports.shape))

BR previous shape: (35955, 10)
BR shape: (35314, 10)


#### Bug Reports Additional Infos

In [89]:
bugreports_add_info_df = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/BR/all_bugs_add_info.csv', sep='|')
print('BugReportsAddInfo.shape: {}'.format(bugreports_add_info_df.shape))
    
bugreports_final = bugreports.set_index('Bug_Number').join(other=bugreports_add_info_df.set_index('Bug_Number'))
bugreports_final.reset_index(inplace=True)

print('Final_BugReports.shape: {}'.format(bugreports_final.shape))

BugReportsAddInfo.shape: (37530, 7)
Final_BugReports.shape: (35314, 16)


#### Bug Reports Names and Descriptions

In [90]:
bugreports_final['br_name'] = bugreports_final.apply(lambda row : 'BR_' + str(row['Bug_Number']) + '_SRC', axis=1)
bugreports_final['br_desc'] = bugreports_final.apply(lambda row : ' '.join([str(el) for el in row]), axis=1) 
bugreports_final.head()

,Bug_Number,Summary,Platform,Component,Version,Creation_Time,Whiteboard,QA_Whiteboard,First_Comment_Text,First_Comment_Creation_Time,Status,Product,Priority,Resolution,Severity,Is_Confirmed,br_name,br_desc
0,506297,Livemarks with null site/feed uris cause sync ...,All,Sync,unspecified,2009-07-24T17:08:43Z,NaN,NaN,2009-07-24 09:54:28 FaultTolerance D...,2009-07-24T17:08:43Z,RESOLVED,Firefox,--,FIXED,normal,True,BR_506297_SRC,506297 Livemarks with null site/feed uris caus...
1,506338,Enhance Crash Recovery to better help the user,All,Session Restore,Trunk,2009-07-24T19:17:21Z,[crashkill][crashkill-metrics],NaN,When our users crash they are pretty much in t...,2009-07-24T19:17:21Z,NEW,Firefox,--,NaN,enhancement,True,BR_506338_SRC,506338 Enhance Crash Recovery to better help t...
2,506507,Dragging multiple bookmarks in the bookmarks s...,x86,Bookmarks & History,Trunk,2009-07-26T06:16:02Z,NaN,NaN,User-Agent: Mozilla/5.0 (Windows; U; Win...,2009-07-26T06:16:02Z,RESOLVED,Firefox,--,WORKSFORME,normal,True,BR_506507_SRC,506507 Dragging multiple bookmarks in the book...
3,506550,Unreliable Back Button navigating nytimes.com,x86,Extension Compatibility,3.5 Branch,2009-07-26T16:12:10Z,[caused by adblock plus][platform-rel-NYTimes],NaN,User-Agent: Mozilla/5.0 (Windows; U; Win...,2009-07-26T16:12:10Z,RESOLVED,Firefox,--,FIXED,normal,False,BR_506550_SRC,506550 Unreliable Back Button navigating nytim...
4,506575,ALT + F4 when dropdown of autocomplete is open...,x86,Address Bar,3.5 Branch,2009-07-26T20:14:54Z,NaN,NaN,Pressing ALT + F4 when the autocomplete dropdo...,2009-07-26T20:14:54Z,NEW,Firefox,P5,NaN,normal,True,BR_506575_SRC,506575 ALT + F4 when dropdown of autocomplete ...


#### Select Subset of Bug Reports for Empirical Study

For the empirical study, made for the creation of the oracle matrix by volunteers, we make a selection of bug reports to filter the most relevant for our analysis and improve the power and improve our conclusions about the studied subject.

In [91]:
brs_versions = ['48 Branch', '49 Branch', '50 Branch', '51 Branch']
brs_status = ['NEW','RESOLVED']
brs_priority = ['P1', 'P2', 'P3']
brs_resolutions = ['FIXED']
brs_severities = ['major', 'normal', 'blocker', 'critical']
brs_isconfirmed = [True]
selected_bugs = bugreports_final[(bugreports_final.Version.isin(brs_versions)) &
                                 (bugreports_final.Status.isin(brs_status)) &
                                 (bugreports_final.Priority.isin(brs_priority)) &
                                 (bugreports_final.Resolution.isin(brs_resolutions)) &
                                 (bugreports_final.Severity.isin(brs_severities)) &
                                 (bugreports_final.Is_Confirmed.isin(brs_isconfirmed))
                                ]
print(selected_bugs.shape)
selected_bugs.head(50)

(39, 18)


,Bug_Number,Summary,Platform,Component,Version,Creation_Time,Whiteboard,QA_Whiteboard,First_Comment_Text,First_Comment_Creation_Time,Status,Product,Priority,Resolution,Severity,Is_Confirmed,br_name,br_desc
9192,1181835,Provide a UI for migrating users' add-ons to w...,Unspecified,Extension Compatibility,49 Branch,2015-07-08T23:01:45Z,[UX] triaged,NaN,We're still not exactly sure how this would wo...,2015-07-08T23:01:45Z,RESOLVED,Firefox,P2,FIXED,normal,True,BR_1181835_SRC,1181835 Provide a UI for migrating users' add-...
11229,1267480,Intermittent browser/components/preferences/in...,Unspecified,Preferences,48 Branch,2016-04-25T23:58:35Z,[stockwell fixed],NaN,https://treeherder.mozilla.org/logviewer.html#...,2016-04-25T23:58:35Z,RESOLVED,Firefox,P3,FIXED,normal,True,BR_1267480_SRC,1267480 Intermittent browser/components/prefer...
11232,1267501,New Private Browsing start-page overflows off ...,Unspecified,Private Browsing,48 Branch,2016-04-26T01:12:11Z,[fxprivacy],NaN,STR: 1. Open a new private browsing window. ...,2016-04-26T01:12:11Z,RESOLVED,Firefox,P1,FIXED,normal,True,BR_1267501_SRC,1267501 New Private Browsing start-page overfl...
11375,1270274,Intermittent browser_share.jsuncaught exceptio...,Unspecified,SocialAPI,49 Branch,2016-05-04T20:55:19Z,NaN,NaN,https://treeherder.mozilla.org/logviewer.html#...,2016-05-04T20:55:19Z,RESOLVED,Firefox,P3,FIXED,normal,True,BR_1270274_SRC,1270274 Intermittent browser_share.jsuncaught ...
11414,1270983,Intermittent browser_contextmenu.jsTest timed ...,Unspecified,Menus,49 Branch,2016-05-06T22:06:20Z,[stockwell fixed],NaN,https://treeherder.mozilla.org/logviewer.html#...,2016-05-06T22:06:20Z,RESOLVED,Firefox,P3,FIXED,normal,True,BR_1270983_SRC,1270983 Intermittent browser_contextmenu.jsTes...
11452,1271607,The new window's size is modified if the last ...,All,General,48 Branch,2016-05-10T12:02:30Z,tpi:+,[good first verify],Created attachment 8750748 Recording #1.mp4 M...,2016-05-10T12:02:30Z,RESOLVED,Firefox,P1,FIXED,normal,True,BR_1271607_SRC,1271607 The new window's size is modified if t...
11695,1276120,Autocomplete suggestions just don't go away af...,Unspecified,Address Bar,48 Branch,2016-05-27T02:34:32Z,[fxsearch],[good first verify] [bugday-20160713],Created attachment 8757112 screenshot 1 - Auto...,2016-05-27T02:34:32Z,RESOLVED,Firefox,P2,FIXED,normal,True,BR_1276120_SRC,1276120 Autocomplete suggestions just don't go...
11824,1278388,Permafailing on the change to firefox50: brows...,Unspecified,Preferences,49 Branch,2016-06-06T21:45:20Z,NaN,NaN,https://treeherder.mozilla.org/logviewer.html#...,2016-06-06T21:45:20Z,RESOLVED,Firefox,P1,FIXED,normal,True,BR_1278388_SRC,1278388 Permafailing on the change to firefox5...
12084,1281190,Keep collecting telemetry for Permissions Noti...,Unspecified,General,49 Branch,2016-06-21T15:20:31Z,[fxprivacy],NaN,POPUP_NOTIFICATION_STATS is expiring soon [1]....,2016-06-21T15:20:31Z,RESOLVED,Firefox,P2,FIXED,normal,True,BR_1281190_SRC,1281190 Keep collecting telemetry for Permissi...
12250,1283542,Automigration from Chrome is broken,Unspecified,Migration,50 Branch,2016-06-30T16:37:26Z,[migration-needs-uplift],NaN,The code and tests expect to deal with profile...,2016-06-30T16:37:26Z,RESOLVED,Firefox,P2,FIXED,normal,True,BR_1283542_SRC,1283542 Automigration from Chrome is broken Un...


## TestCases

#### Test Cases Names and Descriptions

In [92]:
testcases['tc_name'] = testcases.apply(lambda row : 'TC_' + str(row[0]) + '_TRG', axis=1)
testcases['tc_desc'] = testcases.apply(lambda row : ' '.join([str(el) for el in row]), axis=1)
testcases.head()

,TC_Number,TestDay,Feature_ID,Firefox_Feature,Gen_Title,Crt_Nr,Title,Preconditions,Steps,Expected_Result,tc_name,tc_desc
0,1,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,1,Notification - Popup Block,NaN,1. Launch Firefox\n2. Navigate to http://www.p...,1. Firefox is successfully launched\n9. The al...,TC_1_TRG,1 20181221 20 <notificationbox> and <notificat...
1,2,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,2,Notification - Process Hang,NaN,"1. Launch Firefox\n2. In the URL bar, navigate...",1. Firefox is successfully launched\n2. Firefo...,TC_2_TRG,2 20181221 20 <notificationbox> and <notificat...
2,3,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,3,Verify Notifications appear in RTL Mode,NaN,"1. Launch Firefox\n2. In about:config, change ...",1. Firefox is successfully launched\n2.The for...,TC_3_TRG,3 20181221 20 <notificationbox> and <notificat...
3,4,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,4,Verify Notifications appear in High Contrast M...,NaN,"1. While the browser is in High Contrast Mode,...",1. Firefox has been launched.\n2. Firefox begi...,TC_4_TRG,4 20181221 20 <notificationbox> and <notificat...
4,5,20181221,20,<notificationbox> and <notification> changes,<notificationbox> and <notification> changes,5,Verify notifications react to differing Zoom l...,NaN,"1. While the browser is in High Contrast Mode,...",1. Firefox has been launched.\n2. Firefox begi...,TC_5_TRG,5 20181221 20 <notificationbox> and <notificat...


In [93]:
print('Expected instances amount: {} x {} = {}'.format(len(bugreports), len(testcases), len(bugreports) * len(testcases)))

print('Num BRs 48 Branch: {}'.format(len(bugreports[bugreports.Version == '48 Branch'])))
print('Num BRs 49 Branch: {}'.format(len(bugreports[bugreports.Version == '49 Branch'])))
print('Num BRs 50 Branch: {}'.format(len(bugreports[bugreports.Version == '50 Branch'])))
print('Num BRs 51 Branch: {}'.format(len(bugreports[bugreports.Version == '51 Branch'])))

print('Num TCs: {}'.format(len(testcases)))
print('Num BRs: {}'.format(len(bugreports)))

Expected instances amount: 35314 x 207 = 7309998
Num BRs 48 Branch: 412
Num BRs 49 Branch: 353
Num BRs 50 Branch: 518
Num BRs 51 Branch: 461
Num TCs: 207
Num BRs: 35314


## Oracle

#### Estimating Oracle Memory Size

In [94]:
ex_df = pd.DataFrame(index=testcases.tc_name, columns=bugreports_final.br_name, data=0, dtype='int8')
print(ex_df.shape)
print(ex_df.info())
ex_df.head()

(207, 35314)
<class 'pandas.core.frame.DataFrame'>
Index: 207 entries, TC_1_TRG to TC_207_TRG
Columns: 35314 entries, BR_506297_SRC to BR_1516895_SRC
dtypes: int8(35314)
memory usage: 7.0+ MB
None


br_name,BR_506297_SRC,BR_506338_SRC,BR_506507_SRC,BR_506550_SRC,BR_506575_SRC,BR_506729_SRC,BR_506768_SRC,BR_506795_SRC,BR_506820_SRC,BR_506831_SRC,...,BR_1516270_SRC,BR_1516329_SRC,BR_1516358_SRC,BR_1516416_SRC,BR_1516505_SRC,BR_1516547_SRC,BR_1516582_SRC,BR_1516749_SRC,BR_1516792_SRC,BR_1516895_SRC
tc_name,,,,,,,,,,,,,,,,,,,,,
TC_1_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_2_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_3_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_4_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
TC_5_TRG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Oracle Functions and Auxiliary Variables

In [95]:
list_fversion_to_testday = [('48 Branch','20160603'),('48 Branch','20160624'),('48 Branch','20160708'),
                            ('49 Branch','20160722'),('49 Branch','20160812'),('49 Branch','20160826'),
                            ('50 Branch','20160909'),('50 Branch','20160930'),('50 Branch','20161014'),
                            ('51 Branch','20161028'),('51 Branch','20161125'),('51 Branch','20170106')]

NUMBER_SUBSETS = 7

def check_link_condition(br, tc):
    for tup in [(br['Version'],tday) for tday in tc['TestDay'].split(' + ')]:
        if tup in list_fversion_to_testday:
            return True
    return False


def create_links(idx, tc_df, br_df):
    oracle_df = pd.DataFrame(columns=br_df.br_name, index=tc_df.tc_name, data=np.zeros(shape=(len(tc_df),len(br_df))), dtype='int8')
    for idx_1,br in tqdm(br_df.iterrows()):
        for idx_2,tc in tc_df.iterrows():
            if check_link_condition(br, tc):
                oracle_df.at[tc.tc_name, br.br_name] = 1
            else:
                oracle_df.at[tc.tc_name, br.br_name] = 0
    
    oracle_df.to_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/part/trace_matrix_{}.csv'.format(idx))

def create_br_dfs_list():
    list_br_dfs = []
    for i in range(0, 35315, 5045):   # 35315 / 5045 == NUMBER_SUBSETS
        list_br_dfs.append(bugreports_final.iloc[i:i+5045,:])
    return list_br_dfs

def create_tc_dfs_list():
    return [testcases.copy() for i in range(NUMBER_SUBSETS)]

#### Create Small Size Oracle

In [96]:
br_aux = bugreports_final[(bugreports_final.Version == '50 Branch') | (bugreports_final.Version == '60 Branch')].sample(15, random_state=42)
tc_aux = testcases[(testcases.TestDay.str.contains('20161014')) | (testcases.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

br_aux[br_aux.Version == '50 Branch'].loc[:, ['Bug_Number','Version']].head(100)

,Bug_Number,Version
14902,1319983,50 Branch
14763,1318407,50 Branch
12484,1287109,50 Branch
14981,1320548,50 Branch
15367,1325288,50 Branch
12059,1280856,50 Branch


In [97]:
tc_aux[tc_aux.TestDay.str.contains('20161014')].loc[:,['TC_Number','TestDay']].head(100)

,TC_Number,TestDay
18,19,20160603 + 20160624 + 20161014
15,16,20160603 + 20160624 + 20161014
14,15,20160603 + 20160624 + 20161014


In [98]:
create_links('small', tc_aux, br_aux)

small_orc = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/part/trace_matrix_small.csv')
aux_functions.highlight_df(small_orc)

15it [00:00, 594.14it/s]


,tc_name,BR_1441532_SRC,BR_1319983_SRC,BR_1443343_SRC,BR_1464815_SRC,BR_1318407_SRC,BR_1468122_SRC,BR_1445895_SRC,BR_1459431_SRC,BR_1287109_SRC,BR_1320548_SRC,BR_1469153_SRC,BR_1325288_SRC,BR_1463768_SRC,BR_1469753_SRC,BR_1280856_SRC
0,TC_165_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,TC_19_TRG,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1
2,TC_152_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,TC_16_TRG,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1
4,TC_160_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,TC_169_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,TC_145_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,TC_15_TRG,0,1,0,0,1,0,0,0,1,1,0,1,0,0,1
8,TC_149_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,TC_167_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Create Entire Oracle

In [99]:
tasks = [(idx,tc_df, br_df) for idx,(tc_df,br_df) in enumerate(zip(create_tc_dfs_list(),create_br_dfs_list()))]
results = Parallel(n_jobs=7, verbose=3)(delayed(create_links)(idx,tc_df,br_df) for idx,tc_df,br_df in tasks)

[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   3 out of   7 | elapsed:  1.7min remaining:  2.3min


KeyboardInterrupt: 

#### Analyze Oracle Parts Created

In [ ]:
oo_df_2 = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/part/trace_matrix_2.csv')
oo_df_2.set_index('tc_name', inplace=True)

print(oo_df_2.loc['TC_15_TRG', 'BR_1319983_SRC'])
print(oo_df_2.loc['TC_16_TRG', 'BR_1319983_SRC'])
print(oo_df_2.loc['TC_19_TRG', 'BR_1319983_SRC'])

In [ ]:
oo_dfs = []
for i in range(NUMBER_SUBSETS):
    df = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/part/trace_matrix_{}.csv'.format(i))
    oo_dfs.append(df)
    print(df.shape)

#### Join Oracle Parts

In [ ]:
oo_df = pd.DataFrame(index=testcases.tc_name, dtype='int8')
for i in range(NUMBER_SUBSETS):
    aux_df = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/part/trace_matrix_{}.csv'.format(i))
    aux_df.set_index('tc_name', inplace=True)
    oo_df = oo_df.join(aux_df)

print(oo_df.shape)
print(oo_df.info())

## Save DataFrames

In [ ]:
oo_df.to_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/trace_matrix_final.csv')
bugreports.to_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/BR/bugreports_final.csv', index=False)
testcases.to_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/TC/testcases_final.csv', index=False)

# -----

#### Checking Values [0]

#### Analyze Entire Oracle Created

In [ ]:
oo_df_full = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/trace_matrix_final.csv')
oo_df_full.set_index('tc_name', inplace=True)

print(oo_df_full.loc['TC_15_TRG', 'BR_1319983_SRC'])
print(oo_df_full.loc['TC_16_TRG', 'BR_1319983_SRC'])
print(oo_df_full.loc['TC_19_TRG', 'BR_1319983_SRC'])

### -----

#### Checking Values [1]
FVersion to TestDay

In [ ]:
ck_df = pd.DataFrame(columns=['testday','f_version','features_released','testcases_list'])
ck_df.testday = ['20160603', '20160624', '20160708', 
                 '20160722', '20160812', '20160826', 
                 '20160909', '20160930', '20161014', 
                 '20161028', '20161125', '20170106']
ck_df.f_version = ['48 Branch', '48 Branch', '48 Branch', 
                  '49 Branch', '49 Branch', '49 Branch', 
                  '50 Branch', '50 Branch', '50 Branch', 
                  '51 Branch', '51 Branch', '51 Branch' ]
ck_df.features_released = [
    "Awesome Bar Search, Awesome Bar Icons - Left, Awesome Bar Icons - Right",
    "Awesome Bar Search, Awesome Bar Icons - Left, Awesome Bar Icons - Right",
    "apz, Scrolling using different devices (wired mouse, wireless mouse, trackpad/touchpad) - where available devices",
    'context menu - exploratory testing, context menu - full functional testing, pdf viewer, browser customization',
    'windows 10 compatibility, text to speech in reader mode, text to speech on desktop',
    'webgl compatibility, exploratory testing',
    '',
    'Pointer Lock API, WebM EME support for Widevine',
    'New Awesome Bar',
    'Zoom indicator, Downloads dropmaker',
    'WebGL2,  FLAC support,  Indicator for device permissions,  Zoom Indicator',
    'WebGL2, Zoom Indicator, Flash support']

ck_df.testcases_list = ""

included = []
for i,tc in testcases.iterrows():
    for j,row in ck_df.iterrows(): 
        if row['testday'] in tc['TestDay']:
            if ck_df.at[j,'testcases_list'] == "":
                ck_df.at[j,'testcases_list'] = str(tc.TC_Number)
            else:
                ck_df.at[j,'testcases_list'] = ck_df.at[j,'testcases_list'] + " " + str(tc.TC_Number)
            if tc.TC_Number not in included:
                included.append(tc.TC_Number)

ck_df.head(20)

In [ ]:
ck_df.to_csv('../data/mozilla_firefox_v2/firefoxDataset/docs_english/TD_2_FVersion/testday_to_fversion.csv')

#### Analyze Amount of Different Test Cases by Firefox Version

In [ ]:
testcases.TestDay.value_counts()

In [ ]:
testcases.TestDay.value_counts().sum()

In [ ]:
b48_0 = ck_df.loc[0].testcases_list.split(' ')
b48_1 = ck_df.loc[1].testcases_list.split(' ')
b48_2 = ck_df.loc[2].testcases_list.split(' ')
b48 = b48_0 + b48_1 + b48_2
print('Amount Different Test Cases - 48 Branch: {}'.format(len(set(b48))))

b49_0 = ck_df.loc[3].testcases_list.split(' ')
b49_1 = ck_df.loc[4].testcases_list.split(' ')
b49_2 = ck_df.loc[5].testcases_list.split(' ')
b49 = b49_0 + b49_1 + b49_2
print('Amount Different Test Cases - 49 Branch: {}'.format(len(set(b49))))

#b50_0 = ck_df.loc[6].testcases_list.split(' ')
b50_1 = ck_df.loc[7].testcases_list.split(' ')
b50_2 = ck_df.loc[8].testcases_list.split(' ')
b50 = b50_1 + b50_2
print('Amount Different Test Cases - 50 Branch: {}'.format(len(set(b50))))

b51_0 = ck_df.loc[9].testcases_list.split(' ')
b51_1 = ck_df.loc[10].testcases_list.split(' ')
b51_2 = ck_df.loc[11].testcases_list.split(' ')
b51 = b51_0 + b51_1 + b51_2
print('Amount Different Test Cases - 51 Branch: {}'.format(len(set(b51))))

print('Total Amount TCs Sets Union (len(set(b48) | set(b49) | set(b50) | set(b51))): {}'.format(len(set(b48) | set(b49) | set(b50) | set(b51))))

### ---------

#### Checking Values [2]

Checking amount of positive and negative links in oracle

In [ ]:
oracle = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/trace_matrix_final.csv')
oracle.set_index('tc_name', inplace=True, drop=True)

print(oracle.shape)

In [ ]:
num_pos = 0
num_neg = 0

for i in range(len(testcases)):
    counts = oracle.iloc[i, :].value_counts()
    if len(counts) == 2:
        num_neg = num_neg + counts[0]
        num_pos = num_pos + counts[1]
    else:
        num_neg = num_neg + counts[0]
    
print('Num Positive Links: {}'.format(num_pos))
print('Num Negative Links: {}'.format(num_neg))

### ---------

#### Checking Values [3]

Checking subset of oracle

In [ ]:
oracle = pd.read_csv('../data/mozilla_firefox_v2/firefoxDataset/oracle/output/trace_matrix_final.csv')
oracle.set_index('tc_name', inplace=True, drop=True)

print(oracle.shape)

In [ ]:
bugreports_subset_df = bugreports_final[(bugreports_final.Version == '50 Branch') | (bugreports_final.Version == '60 Branch')].sample(15, random_state=42)
bugreports_subset_df[bugreports_subset_df.Version == '50 Branch'].loc[:, ['Bug_Number','Version']].head(100)

In [ ]:
testcases_subset_df = testcases[(testcases.TestDay.str.contains('20161014')) | (testcases.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13,14,15,16,17,18,19,20,21]]  # should link with 50 Branch
aux_tc = testcases[testcases.tc_name.isin(selected_testcases)]

tc_subset_df = testcases_subset_df.append(aux_tc)
tc_subset_df.drop_duplicates(inplace=True)

tc_subset_df[tc_subset_df.TestDay.str.contains('20161014')].loc[:,['TC_Number','TestDay']].head(100)

In [ ]:
testcases_names_subset = tc_subset_df.tc_name
bug_reports_names_subset = bugreports_subset_df.br_name
orc_subset_df = oracle.loc[testcases_names_subset, bug_reports_names_subset]

print('TestCases Subset Shape: {}'.format(tc_subset_df.shape))
print('BugReports Subset Shape: {}'.format(bugreports_subset_df.shape))
print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

In [ ]:
aux_functions.highlight_df(orc_subset_df)